# Earnings Call Project: Data Cleaning
<br>
CIS 831 Deep Learning – Term Project<br>
Kansas State University
<br><br>
James Chapman<br>
John Woods<br>
Nathan Diehl<br>
<br>

This notebook uses the transformer architecture from the [HTML paper](https://www.researchgate.net/publication/340385140_HTML_Hierarchical_Transformer-based_Multi-task_Learning_for_Volatility_Prediction). 

From the previous notebooks, we have features for both audio and text, of each sentence, of each meeting.<br>
- audio 
    - PRAAT (27 features)
- text
    - GLOVE (300 features)
    - ROBERTA (1024 features)
    - ROBERTA with averaging (1024 features)
    - FinLang investopedia (768 features) from huggingface sentencetransformers
    - BGE financial (1024 features) from huggingface sentencetransformers

This notebook performs 3 nested loops. Each audio/text pair, and for each N_DAYS, we train models with 17 different alpha values. The alpha value the lowest MSE, based on the validation set, is been used to train a model with Both training and validation sets. This model is finally tested on the test set.

These notebooks also give us a playground to work with the data, and test the models. Now we can insert sentiment detection, segmentation of the text, etc.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as F
from torch import nn
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(777)
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
# static hyper-parameters from paper
max_norm = 1.0
num_epochs = 10
warmup_steps = 1000
batch_size=4
dropout=0.5
heads=2
depth=2

lr=0.000002 # 1/10 learning rate of paper

all_n_days = ['3', '7', '15', '30']
alphas = [0.1, 0.3, 0.5, 0.7, 0.9]
is_using_scaler = True # whether or not to use standardscaler

# This is the hierarchical transformer from the HTML paper.

## This is taken directly from the GitHub account for HTML paper.
- a few adaptions are highlighted with ##############################################


- [GitHub HTML - util.py](https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/util/util.py)
- [GitHub HTML - transformers_gpu.py](https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/transformers_gpu.py)

In [4]:

# https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/transformers_gpu.py
def mask_(matrices, maskval=0.0, mask_diagonal=True):
    """
    Masks out all values in the given batch of matrices where i <= j holds,
    i < j if mask_diagonal is false

    In place operation

    :param tns:
    :return:
    """

    b, h, w = matrices.size()

    indices = torch.triu_indices(h, w, offset=0 if mask_diagonal else 1)
    matrices[:, indices, indices[1]] = maskval

def contains_nan(tensor):
    return bool((tensor != tensor).sum() > 0)

# https://github.com/YangLinyi/HTML-Hierarchical-Transformer-based-Multi-task-Learning-for-Volatility-Prediction/blob/master/Model/Sentence-Level-Transformer/transformers/transformers_gpu.py
class SelfAttention(nn.Module):
    def __init__(self, emb, heads=8, mask=False):
        """
        :param emb:
        :param heads:
        :param mask:
        """

        super().__init__()

        self.emb = emb
        self.heads = heads
        self.mask = mask

        self.tokeys = nn.Linear(emb, emb * heads, bias=False)
        self.toqueries = nn.Linear(emb, emb * heads, bias=False)
        self.tovalues = nn.Linear(emb, emb * heads, bias=False)

        self.unifyheads = nn.Linear(heads * emb, emb)

    def forward(self, x):

        b, t, e = x.size()
        h = self.heads
        assert e == self.emb

        keys    = self.tokeys(x)   .view(b, t, h, e)
        queries = self.toqueries(x).view(b, t, h, e)
        values  = self.tovalues(x) .view(b, t, h, e)

        # compute scaled dot-product self-attention

        # - fold heads into the batch dimension
        keys = keys.transpose(1, 2).contiguous().view(b * h, t, e)
        queries = queries.transpose(1, 2).contiguous().view(b * h, t, e)
        values = values.transpose(1, 2).contiguous().view(b * h, t, e)

        queries = queries / (e ** (1/4))
        keys    = keys / (e ** (1/4))
        # - Instead of dividing the dot products by sqrt(e), we scale the keys and values.
        #   This should be more memory efficient

        # - get dot product of queries and keys, and scale
        dot = torch.bmm(queries, keys.transpose(1, 2))

        assert dot.size() == (b*h, t, t)

        if self.mask: # mask out the lower half of the dot matrix,including the diagonal
            mask_(dot, maskval=float('-inf'), mask_diagonal=False)

        dot = F.softmax(dot, dim=2) # dot now has row-wise self-attention probabilities

        ##############################################
        ##############################################
        # assert not util.contains_nan(dot[:, 1:, :]) # only the forst row may contain nan
        assert not contains_nan(dot[:, 1:, :]) # only the forst row may contain nan 
        ##############################################
        ##############################################
        
        if self.mask == 'first':
            dot = dot.clone()
            dot[:, :1, :] = 0.0
            # - The first row of the first attention matrix is entirely masked out, so the softmax operation results
            #   in a division by zero. We set this row to zero by hand to get rid of the NaNs

        # apply the self attention to the values
        out = torch.bmm(dot, values).view(b, h, t, e)

        # swap h, t back, unify heads
        out = out.transpose(1, 2).contiguous().view(b, t, h * e)

        return self.unifyheads(out)

class TransformerBlock(nn.Module):
    def __init__(self, emb, heads, mask, seq_length, ff_hidden_mult=4, dropout=0.0):
        super().__init__()

        self.attention = SelfAttention(emb, heads=heads, mask=mask)
        self.mask = mask

        self.norm1 = nn.LayerNorm(emb)
        self.norm2 = nn.LayerNorm(emb)

        self.ff = nn.Sequential(
            nn.Linear(emb, ff_hidden_mult * emb),
            nn.ReLU(),
            nn.Linear(ff_hidden_mult * emb, emb)
        )

        self.do = nn.Dropout(dropout)

    def forward(self, x):

        attended = self.attention(x)

        x = self.norm1(attended + x)

        x = self.do(x)

        fedforward = self.ff(x)

        x = self.norm2(fedforward + x)

        x = self.do(x)

        return x

class RTransformer(nn.Module):
    """
    Transformer for sequences Regression    
    
    """

    def __init__(self, emb, heads, depth, seq_length, num_tokens, num_classes, max_pool=True, dropout=0.0):
        """
        emb: Embedding dimension
        heads: nr. of attention heads
        depth: Number of transformer blocks
        seq_length: Expected maximum sequence length
        num_tokens: Number of tokens (usually words) in the vocabulary
        num_classes: Number of classes.
        max_pool: If true, use global max pooling in the last layer. If false, use global
                         average pooling.
        """
        super().__init__()

        self.num_tokens, self.max_pool = num_tokens, max_pool

        #self.token_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=num_tokens)
        self.pos_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=seq_length)

        tblocks = []
        for i in range(depth):
            tblocks.append(
                TransformerBlock(emb=emb, heads=heads, seq_length=seq_length, mask=False, dropout=dropout))

        self.tblocks = nn.Sequential(*tblocks)

        self.toprobs = nn.Linear(emb, num_classes)
        self.toprobs_b = nn.Linear(emb, num_classes)
        self.do = nn.Dropout(dropout)

    def forward(self, x):
        """
        :param x: A batch by sequence length integer tensor of token indices.
        :return: predicted log-probability vectors for each token based on the preceding tokens.
        """
        sentences_emb = x
        b, t, e = x.size()

        ##############################################
        ##############################################
        # swap d() for device
        # positions = self.pos_embedding(torch.arange(t, device=d()))[None, :, :].expand(b, t, e)
        positions = self.pos_embedding(torch.arange(t, device=device))[None, :, :].expand(b, t, e)
        ##############################################
        ##############################################
        #positions = self.pos_embedding(torch.arange(t))[None, :, :].expand(b, t, e)
        #positions = torch.tensor(positions, dtype=torch.float32)
        x = sentences_emb.cuda() + positions
        x = self.do(x)

        x = self.tblocks(x)

        x = x.max(dim=1)[0] if self.max_pool else x.mean(dim=1) # pool over the time dimension
        
        
        x_a = self.toprobs(x)
        x_b = self.toprobs_b(x)
        x_a = torch.squeeze(x_a)
        x_b = torch.squeeze(x_b)
        #print('x shape: ',x.shape)
        return x_a, x_b


# dataset builder and training loop

In [5]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, embeddings, labels, labels_b):
        self.embeddings = embeddings
        self.labels = labels
        self.labels_b = labels_b

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        emb = self.embeddings[idx]
        label = self.labels[idx]
        label_b = self.labels_b[idx]
        return emb, label, label_b 

def get_data(data_directory, n_days, year, no_validation_set, is_using_scaler, batch_size):
    # features
    train_features = np.load('data/{}/train_features{}.npy'.format(data_directory, year))
    val_features = np.load('data/{}/val_features{}.npy'.format(data_directory, year))
    test_features = np.load('data/{}/test_features{}.npy'.format(data_directory, year))
    # targets (n_days volatility)
    train_targets = np.load('data/{}/train_targets{}_{}.npy'.format(data_directory, year, n_days))
    val_targets = np.load('data/{}/val_targets{}_{}.npy'.format(data_directory, year, n_days))
    test_targets = np.load('data/{}/test_targets{}_{}.npy'.format(data_directory, year, n_days))
    # secondary targets (log percent change of day n)
    train_secondary_targets = np.load('data/{}/train_secondary_targets{}_{}.npy'.format(data_directory, year, n_days))
    val_secondary_targets = np.load('data/{}/val_secondary_targets{}_{}.npy'.format(data_directory, year, n_days))
    test_secondary_targets = np.load('data/{}/test_secondary_targets{}_{}.npy'.format(data_directory, year, n_days))

    train_secondary_targets[np.isneginf(train_secondary_targets)] = 0
    val_secondary_targets[np.isneginf(val_secondary_targets)] = 0
    test_secondary_targets[np.isneginf(test_secondary_targets)] = 0

    # after hyperparameters are tuned on the validation set
    # add validation set to training set
    if no_validation_set:
        train_features = np.concatenate((train_features, val_features), axis=0)
        train_targets = np.concatenate((train_targets, val_targets), axis=0)
        train_secondary_targets = np.concatenate((train_secondary_targets, val_secondary_targets), axis=0)

    if is_using_scaler:
        # Scaling features
        feature_scaler = StandardScaler()
        train_features = feature_scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
        val_features = feature_scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
        test_features = feature_scaler.transform(test_features.reshape(-1, test_features.shape[-1])).reshape(test_features.shape)
        # Scaling primary targets
        target_scaler = StandardScaler()
        train_targets = target_scaler.fit_transform(train_targets.reshape(-1, 1)).reshape(train_targets.shape)
        val_targets = target_scaler.transform(val_targets.reshape(-1, 1)).reshape(val_targets.shape)
        test_targets = target_scaler.transform(test_targets.reshape(-1, 1)).reshape(test_targets.shape)
        # Scaling secondary targets
        secondary_target_scaler = StandardScaler()
        train_secondary_targets = secondary_target_scaler.fit_transform(train_secondary_targets.reshape(-1, 1)).reshape(train_secondary_targets.shape)
        val_secondary_targets = secondary_target_scaler.transform(val_secondary_targets.reshape(-1, 1)).reshape(val_secondary_targets.shape)
        test_secondary_targets = secondary_target_scaler.transform(test_secondary_targets.reshape(-1, 1)).reshape(test_secondary_targets.shape)

    # Dataset & DataLoader
    training_set = Dataset(train_features, train_targets, train_secondary_targets) 
    val_set = Dataset(val_features, val_targets, val_secondary_targets)
    test_set = Dataset(test_features, test_targets, test_secondary_targets)
    trainloader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, num_workers=0)
    valloader = torch.utils.data.DataLoader(val_set, batch_size=len(val_set), shuffle=False, num_workers=0)
    testloader = torch.utils.data.DataLoader(test_set, batch_size=len(test_set), shuffle=False, num_workers=0)

    print(train_features.shape, train_targets.shape, train_secondary_targets.shape) 
    print(val_features.shape, val_targets.shape, val_secondary_targets.shape) 
    print(test_features.shape, test_targets.shape, test_secondary_targets.shape) 

    # train_features.shape[2] is the number of features, need to instantiate the model
    # need target scaler to inverse transform
    return trainloader, valloader, testloader, train_features.shape[2], target_scaler 


In [6]:
# this training loop is actually reused for training, validation, and testing
def  training_loop(trainloader, loader, emb_dimensions, heads, depth,dropout, warmup_steps, 
                   batch_size, num_epochs, alphas, max_norm, lr, run_name, target_scaler):
    alphas_results = []
    for alpha in alphas:
        model = RTransformer(emb=emb_dimensions, heads=heads, depth=depth, seq_length=523, 
                            num_tokens=4000, num_classes=1, max_pool=False, dropout=dropout).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=lr)
        # Linear LR warmup for the first #warmup_steps training examples
        scheduler = LambdaLR(opt, lr_lambda=lambda step: min(1.0, step / (warmup_steps/batch_size)))

        seen = 0
        val_loss_a_history =  [] # keep track of the loss for the current Alpha value
        writer = SummaryWriter(log_dir= f'KeFVP_runs/{run_name}_{alpha}')
        progress_bar = tqdm(range(num_epochs), desc="Training Progress", unit="epoch")
        for epoch in progress_bar:
            model.train()
            train_loss_total = 0
            for i, (inputs, labels, labels_b) in enumerate(trainloader): #training data
                seen += inputs.size(0)
                inputs = inputs.to(device, dtype=torch.float32)
                labels = labels.to(device, dtype=torch.float32)
                labels_b = labels_b.to(device, dtype=torch.float32)
                out_a, out_b = model(inputs)
                out_a = out_a.view(-1)
                out_b = out_b.view(-1)

                # Compute the combined loss (multitask)
                loss_a = F.mse_loss(out_a, labels)
                loss_b = F.mse_loss(out_b, labels_b)
                loss = alpha * loss_a + (1 - alpha) * loss_b # Alpha parameter
            
                opt.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
                opt.step()
                if seen < warmup_steps:
                    scheduler.step()

                # we only care about the primary target
                # although we are training with the scaled loss, reporting the volatility must be unscaled
                out_a_unscaled = target_scaler.inverse_transform(out_a.detach().cpu().numpy().reshape(-1, 1)).reshape(out_a.shape)
                labels_unscaled = target_scaler.inverse_transform(labels.detach().cpu().numpy().reshape(-1, 1)).reshape(labels.shape)
                loss_a_unscaled = F.mse_loss(torch.tensor(out_a_unscaled, device=device), torch.tensor(labels_unscaled, device=device))
                train_loss_total += loss_a_unscaled.item() * inputs.size(0) # MSE of volatility training (multiplied by current batch size)

            # Epoch average of unscaled training loss
            train_loss_avg = train_loss_total / len(trainloader.dataset)
            
            # Epoch Evaluation
            model.eval()
            val_loss_a_total = 0.0
            with torch.no_grad():
                for i, (inputs, labels, labels_b) in enumerate(loader): #validation or testing
                    inputs = inputs.to(device, dtype=torch.float32)
                    labels = labels.to(device, dtype=torch.float32)
                    labels_b = labels_b.to(device, dtype=torch.float32)

                    out_a, out_b = model(inputs)
                    out_a = out_a.view(-1)
                    
                    out_a_unscaled = target_scaler.inverse_transform(out_a.detach().cpu().numpy().reshape(-1, 1)).reshape(out_a.shape)
                    labels_unscaled = target_scaler.inverse_transform(labels.detach().cpu().numpy().reshape(-1, 1)).reshape(labels.shape)
                    loss_a_unscaled = F.mse_loss(torch.tensor(out_a_unscaled, device=device), torch.tensor(labels_unscaled, device=device))
                    val_loss_a_total += loss_a_unscaled.item() * inputs.size(0) # MSE of volatility

                val_loss_a_avg = val_loss_a_total / len(loader.dataset) # MSE of volatility of Val/test!!
            # Epoch finished
            writer.add_scalar('Train Loss/train', train_loss_avg, epoch)
            writer.add_scalar('Loss A/val', val_loss_a_avg, epoch)
            # Update progress bar description and postfix
            progress_bar.set_description(f"Epoch {epoch+1}/{num_epochs}")
            progress_bar.set_postfix({
                "Train Loss": f"{train_loss_avg:.3f}",
                "Test Loss A": f"{val_loss_a_avg:.3f}"
            })
            # for alpha optimization, collect each Epoch's average loss
            val_loss_a_history.append(val_loss_a_avg)
        # all Epochs finished for this alpha value
        min_loss = round(min(val_loss_a_history), 3)
        alphas_results.append((alpha, min_loss))
        writer.close()
    print(alphas_results)
    best_alpha, lowest_val_loss = min(alphas_results, key=lambda x: x[1])
    return best_alpha, lowest_val_loss # if testING, this is the MSE  of the test set

In [7]:

def train_val_test(data_directory, n_days, year):
    run_name = data_directory + year + '_' + n_days 
    # using training set and validation set, determine optimum hyper-parameters (just alpha)
    no_validation_set = False
    trainloader,valloader,testloader,emb_dimensions, target_scaler = get_data(data_directory, n_days, year, no_validation_set, is_using_scaler, batch_size)
    best_alpha, _ = training_loop(trainloader, valloader, emb_dimensions, heads, depth,dropout, warmup_steps, 
                            batch_size, num_epochs, alphas, max_norm, lr, run_name, target_scaler)
    # using optimum alpha, retrain with all training/validation sets and test on testset
    test_run_name = run_name + '_test'
    no_validation_set = True
    trainloader,valloader,testloader,emb_dimensions, target_scaler = get_data(data_directory, n_days, year, no_validation_set, is_using_scaler, batch_size)
    _, MSE_testset = training_loop(trainloader, testloader, emb_dimensions, heads, depth, dropout, warmup_steps, 
                        batch_size, num_epochs, [best_alpha], max_norm, lr, test_run_name, target_scaler)

    print('----------------------------------------')
    print(f'run_name-----------{run_name}')
    print(f'best_alpha---------{best_alpha}')
    print(f'MSE_testset--------{MSE_testset}')
    print('----------------------------------------')
    return best_alpha, MSE_testset

# Train & Test original dataset

In [8]:


directories = ['KeFVP_emotion2vec_Roberta', 'KeFVP_emotion2vec_Roberta2', 'KeFVP_emotion2vec_investopedia', 'KeFVP_emotion2vec_bge', 'KeFVP_emotion2vec_bge_base', 'KeFVP_emotion2vec_glove']#

final_results = []
for data_directory in directories: # different feature engineering features
    for n_days in all_n_days: 
        year = '' # for the original data we don't separate by years (this is for Maec)
        best_alpha, MSE_testset = train_val_test(data_directory, n_days, year)
        final_results.append([data_directory, n_days, best_alpha, MSE_testset])

(391, 523, 1060) (391,) (391,)
(56, 523, 1060) (56,) (56,)
(111, 523, 1060) (111,) (111,)


Epoch 10/10: 100%|██████████| 10/10 [00:24<00:00,  2.46s/epoch, Train Loss=0.665, Test Loss A=0.685]


[(0.1, 0.557), (0.3, 0.549), (0.5, 0.566), (0.7, 0.547), (0.9, 0.54)]
(447, 523, 1060) (447,) (447,)
(56, 523, 1060) (56,) (56,)
(111, 523, 1060) (111,) (111,)


Epoch 10/10: 100%|██████████| 10/10 [00:29<00:00,  2.93s/epoch, Train Loss=0.659, Test Loss A=0.701]


[(0.9, 0.697)]
----------------------------------------
run_name-----------KeFVP_emotion2vec_Roberta_3
best_alpha---------0.9
MSE_testset--------0.697
----------------------------------------
(391, 523, 1060) (391,) (391,)
(56, 523, 1060) (56,) (56,)
(111, 523, 1060) (111,) (111,)


...

[(0.9, 0.255)]
----------------------------------------
run_name-----------KeFVP_emotion2vec_glove_15
best_alpha---------0.9
MSE_testset--------0.255
----------------------------------------
(391, 523, 336) (391,) (391,)
(56, 523, 336) (56,) (56,)
(111, 523, 336) (111,) (111,)


Epoch 10/10: 100%|██████████| 10/10 [00:06<00:00,  1.58epoch/s, Train Loss=0.183, Test Loss A=0.136]


[(0.1, 0.129), (0.3, 0.133), (0.5, 0.128), (0.7, 0.138), (0.9, 0.129)]
(447, 523, 336) (447,) (447,)
(56, 523, 336) (56,) (56,)
(111, 523, 336) (111,) (111,)


Epoch 10/10: 100%|██████████| 10/10 [00:07<00:00,  1.38epoch/s, Train Loss=0.176, Test Loss A=0.214]

[(0.5, 0.208)]
----------------------------------------
run_name-----------KeFVP_emotion2vec_glove_30
best_alpha---------0.5
MSE_testset--------0.208
----------------------------------------


In [9]:
final_results = pd.DataFrame(final_results, columns=['Embedding Variant', 'n_days', 'best_alpha', 'MSE_testset'])
final_results.to_csv('data/KeFVP_emotion2vec_final_results.csv', index=False)
final_results

,Embedding Variant,n_days,best_alpha,MSE_testset
0,KeFVP_emotion2vec_Roberta,3,0.9,0.697
1,KeFVP_emotion2vec_Roberta,7,0.7,0.340
2,KeFVP_emotion2vec_Roberta,15,0.7,0.224
3,KeFVP_emotion2vec_Roberta,30,0.5,0.168
4,KeFVP_emotion2vec_Roberta2,3,0.7,0.723
5,KeFVP_emotion2vec_Roberta2,7,0.9,0.329
6,KeFVP_emotion2vec_Roberta2,15,0.9,0.245
7,KeFVP_emotion2vec_Roberta2,30,0.3,0.176
8,KeFVP_emotion2vec_investopedia,3,0.5,0.675
9,KeFVP_emotion2vec_investopedia,7,0.9,0.335


In [10]:
final_results = final_results.drop(columns=['best_alpha'])
final_results = final_results.pivot(index='Embedding Variant', columns='n_days', values='MSE_testset')
final_results

n_days,15,3,30,7
Embedding Variant,,,,
KeFVP_emotion2vec_Roberta,0.224,0.697,0.168,0.340
KeFVP_emotion2vec_Roberta2,0.245,0.723,0.176,0.329
KeFVP_emotion2vec_bge,0.221,0.678,0.179,0.346
KeFVP_emotion2vec_bge_base,0.233,0.711,0.191,0.329
KeFVP_emotion2vec_glove,0.255,0.717,0.208,0.372
KeFVP_emotion2vec_investopedia,0.234,0.675,0.192,0.335


# Train & Test MAEC dataset

# 2015

In [8]:
directories = ['KeFVP_emotion2vec_MAEC15_Roberta', 'KeFVP_emotion2vec_MAEC15_Roberta2', 'KeFVP_emotion2vec_MAEC15_investopedia', 'KeFVP_emotion2vec_MAEC15_bge', 'KeFVP_emotion2vec_MAEC15_bge_base', 'KeFVP_emotion2vec_MAEC15_glove']#
#years = ['_2015', '_2016', '_2017']

KeFVP_MAEC15_final_results = []
for data_directory in directories: # different feature engineering features
    for n_days in all_n_days: 
        #for year in years:
        year = ''
        best_alpha, MSE_testset = train_val_test(data_directory, n_days, year)
        KeFVP_MAEC15_final_results.append([data_directory, n_days, year, best_alpha, MSE_testset])

(535, 500, 1060) (535,) (535,)
(154, 500, 1060) (154,) (154,)
(154, 500, 1060) (154,) (154,)


Epoch 10/10: 100%|██████████| 10/10 [00:33<00:00,  3.38s/epoch, Train Loss=0.462, Test Loss A=0.947]


[(0.1, 0.544), (0.3, 0.511), (0.5, 0.515), (0.7, 0.548), (0.9, 0.509)]
(689, 500, 1060) (689,) (689,)
(154, 500, 1060) (154,) (154,)
(154, 500, 1060) (154,) (154,)


Epoch 10/10: 100%|██████████| 10/10 [00:42<00:00,  4.26s/epoch, Train Loss=0.485, Test Loss A=1.188]


[(0.9, 0.509)]
----------------------------------------
run_name-----------KeFVP_emotion2vec_MAEC15_Roberta_3
best_alpha---------0.9
MSE_testset--------0.509
----------------------------------------
(535, 500, 1060) (535,) (535,)
(154, 500, 1060) (154,) (154,)
(154, 500, 1060) (154,) (154,)


...

[(0.7, 0.19)]
----------------------------------------
run_name-----------KeFVP_emotion2vec_MAEC15_glove_15
best_alpha---------0.7
MSE_testset--------0.19
----------------------------------------
(535, 500, 336) (535,) (535,)
(154, 500, 336) (154,) (154,)
(154, 500, 336) (154,) (154,)


Epoch 10/10: 100%|██████████| 10/10 [00:08<00:00,  1.18epoch/s, Train Loss=0.151, Test Loss A=0.178]


[(0.1, 0.172), (0.3, 0.169), (0.5, 0.167), (0.7, 0.177), (0.9, 0.162)]
(689, 500, 336) (689,) (689,)
(154, 500, 336) (154,) (154,)
(154, 500, 336) (154,) (154,)


Epoch 10/10: 100%|██████████| 10/10 [00:10<00:00,  1.09s/epoch, Train Loss=0.156, Test Loss A=0.174]

[(0.9, 0.162)]
----------------------------------------
run_name-----------KeFVP_emotion2vec_MAEC15_glove_30
best_alpha---------0.9
MSE_testset--------0.162
----------------------------------------


In [16]:
KeFVP_MAEC15_final_results = pd.DataFrame(KeFVP_MAEC15_final_results, columns=['Embedding Variant', 'n_days', 'year', 'best_alpha', 'MSE_testset'])
KeFVP_MAEC15_final_results.to_csv('data/KeFVP_emotion2vec_MAEC15_final_results.csv', index=False)
KeFVP_MAEC15_final_results

,Embedding Variant,n_days,year,best_alpha,MSE_testset
0,KeFVP_emotion2vec_MAEC15_Roberta,3,,0.9,0.509
1,KeFVP_emotion2vec_MAEC15_Roberta,7,,0.3,0.281
2,KeFVP_emotion2vec_MAEC15_Roberta,15,,0.5,0.186
3,KeFVP_emotion2vec_MAEC15_Roberta,30,,0.5,0.154
4,KeFVP_emotion2vec_MAEC15_Roberta2,3,,0.7,0.514
5,KeFVP_emotion2vec_MAEC15_Roberta2,7,,0.3,0.272
6,KeFVP_emotion2vec_MAEC15_Roberta2,15,,0.3,0.183
7,KeFVP_emotion2vec_MAEC15_Roberta2,30,,0.5,0.153
8,KeFVP_emotion2vec_MAEC15_investopedia,3,,0.7,0.508
9,KeFVP_emotion2vec_MAEC15_investopedia,7,,0.9,0.267


In [10]:

KeFVP_MAEC15_final_results = KeFVP_MAEC15_final_results.drop(columns=['best_alpha', 'year'])
KeFVP_MAEC15_final_results = KeFVP_MAEC15_final_results.pivot(index='Embedding Variant', columns='n_days', values='MSE_testset')
KeFVP_MAEC15_final_results

n_days,15,3,30,7
Embedding Variant,,,,
KeFVP_emotion2vec_MAEC15_Roberta,0.186,0.509,0.154,0.281
KeFVP_emotion2vec_MAEC15_Roberta2,0.183,0.514,0.153,0.272
KeFVP_emotion2vec_MAEC15_bge,0.179,0.497,0.159,0.263
KeFVP_emotion2vec_MAEC15_bge_base,0.193,0.503,0.150,0.263
KeFVP_emotion2vec_MAEC15_glove,0.190,0.530,0.162,0.283
KeFVP_emotion2vec_MAEC15_investopedia,0.184,0.508,0.153,0.267


# 2016

In [8]:
directories = ['KeFVP_emotion2vec_MAEC16_Roberta', 'KeFVP_emotion2vec_MAEC16_Roberta2', 'KeFVP_emotion2vec_MAEC16_investopedia', 'KeFVP_emotion2vec_MAEC16_bge', 'KeFVP_emotion2vec_MAEC16_bge_base', 'KeFVP_emotion2vec_MAEC16_glove']#
#years = ['_2015', '_2016', '_2017']

KeFVP_MAEC16_final_results = []
for data_directory in directories: # different feature engineering features
    for n_days in all_n_days: 
        #for year in years:
        year = ''
        best_alpha, MSE_testset = train_val_test(data_directory, n_days, year)
        KeFVP_MAEC16_final_results.append([data_directory, n_days, year, best_alpha, MSE_testset])

(980, 500, 1060) (980,) (980,)
(280, 500, 1060) (280,) (280,)
(280, 500, 1060) (280,) (280,)


Epoch 10/10: 100%|██████████| 10/10 [01:01<00:00,  6.14s/epoch, Train Loss=0.500, Test Loss A=0.657]


[(0.1, 0.362), (0.3, 0.37), (0.5, 0.368), (0.7, 0.364), (0.9, 0.365)]
(1260, 500, 1060) (1260,) (1260,)
(280, 500, 1060) (280,) (280,)
(280, 500, 1060) (280,) (280,)


Epoch 10/10: 100%|██████████| 10/10 [01:17<00:00,  7.77s/epoch, Train Loss=0.483, Test Loss A=0.413]


[(0.1, 0.363)]
----------------------------------------
run_name-----------KeFVP_emotion2vec_MAEC16_Roberta_3
best_alpha---------0.1
MSE_testset--------0.363
----------------------------------------
(980, 500, 1060) (980,) (980,)
(280, 500, 1060) (280,) (280,)
(280, 500, 1060) (280,) (280,)


...

[(0.7, 0.163)]
----------------------------------------
run_name-----------KeFVP_emotion2vec_MAEC16_glove_15
best_alpha---------0.7
MSE_testset--------0.163
----------------------------------------
(980, 500, 336) (980,) (980,)
(280, 500, 336) (280,) (280,)
(280, 500, 336) (280,) (280,)


Epoch 10/10: 100%|██████████| 10/10 [00:16<00:00,  1.62s/epoch, Train Loss=0.190, Test Loss A=0.142]


[(0.1, 0.139), (0.3, 0.138), (0.5, 0.138), (0.7, 0.137), (0.9, 0.137)]
(1260, 500, 336) (1260,) (1260,)
(280, 500, 336) (280,) (280,)
(280, 500, 336) (280,) (280,)


Epoch 10/10: 100%|██████████| 10/10 [00:19<00:00,  1.93s/epoch, Train Loss=0.179, Test Loss A=0.139]

[(0.7, 0.138)]
----------------------------------------
run_name-----------KeFVP_emotion2vec_MAEC16_glove_30
best_alpha---------0.7
MSE_testset--------0.138
----------------------------------------


In [9]:
KeFVP_MAEC16_final_results = pd.DataFrame(KeFVP_MAEC16_final_results, columns=['Embedding Variant', 'n_days', 'year', 'best_alpha', 'MSE_testset'])
KeFVP_MAEC16_final_results.to_csv('data/KeFVP_emotion2vec_MAEC16_final_results.csv', index=False)
KeFVP_MAEC16_final_results

,Embedding Variant,n_days,year,best_alpha,MSE_testset
0,KeFVP_emotion2vec_MAEC16_Roberta,3,,0.1,0.363
1,KeFVP_emotion2vec_MAEC16_Roberta,7,,0.9,0.211
2,KeFVP_emotion2vec_MAEC16_Roberta,15,,0.5,0.157
3,KeFVP_emotion2vec_MAEC16_Roberta,30,,0.5,0.135
4,KeFVP_emotion2vec_MAEC16_Roberta2,3,,0.1,0.347
5,KeFVP_emotion2vec_MAEC16_Roberta2,7,,0.5,0.200
6,KeFVP_emotion2vec_MAEC16_Roberta2,15,,0.3,0.160
7,KeFVP_emotion2vec_MAEC16_Roberta2,30,,0.3,0.129
8,KeFVP_emotion2vec_MAEC16_investopedia,3,,0.3,0.352
9,KeFVP_emotion2vec_MAEC16_investopedia,7,,0.7,0.195


In [10]:

KeFVP_MAEC16_final_results = KeFVP_MAEC16_final_results.drop(columns=['best_alpha', 'year'])
KeFVP_MAEC16_final_results = KeFVP_MAEC16_final_results.pivot(index='Embedding Variant', columns='n_days', values='MSE_testset')
KeFVP_MAEC16_final_results

n_days,15,3,30,7
Embedding Variant,,,,
KeFVP_emotion2vec_MAEC16_Roberta,0.157,0.363,0.135,0.211
KeFVP_emotion2vec_MAEC16_Roberta2,0.160,0.347,0.129,0.200
KeFVP_emotion2vec_MAEC16_bge,0.146,0.350,0.122,0.200
KeFVP_emotion2vec_MAEC16_bge_base,0.157,0.366,0.127,0.205
KeFVP_emotion2vec_MAEC16_glove,0.163,0.363,0.138,0.223
KeFVP_emotion2vec_MAEC16_investopedia,0.145,0.352,0.130,0.195


In [11]:
%load_ext tensorboard
%tensorboard --logdir KeFVP_runs 

In [12]:
:)

SyntaxError: unmatched ')' (1896645534.py, line 1)